# Calculating Scheduled Headways for CTA Buses - work in progress

### What This Does

TBD

## In progress

- Calculate scheduled headways at each stop, making sure they make sense given different patterns that
run at different times of day for each route


## To Do Next

TBD

## Notes

TBD


## Strategy for Scheduled Headways (Tentative)

1. Which service IDs serve a given stop?

2. When are the earliest/latest times those service IDs are in effect?

3. What are the headways within each timeframe?

## Needs Fixing
 
TBD

In [51]:
import requests
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from shapely import Point, LineString
import datetime as dt
import numpy as np
from static_gtfs_analysis import *

ModuleNotFoundError: No module named 'actual_headways'

# Get Scheduled Stop Times and Headways


In [3]:
# Use Laurie's code to get gtfs feed data
gtfs_feed = download_extract_format('20230105')

INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230105
Loading shapes.txt: 100%|██████████| 7/7 [00:04<00:00,  1.56it/s]


In [2]:
%%capture --no-display
# turn off warnings

######### Scratch
gtfs_version_id = '20230105'
route_id = '55'
service_date_string = '2023-01-09'
# start_timedelta_string ='02:30'
# end_timedelta_string = '26:30'


In [48]:
# Use Laurie's code to get gtfs feed data
gtfs_feed = download_extract_format(gtfs_version_id)

INFO:root:Downloading CTA data


In [49]:
def string_to_datetime(date_string:str) -> pendulum.datetime:
        '''Parameters:\n
        date_string is in the format "YYYY-MM-DD" obtained using get_headways().\n
        Data returned:\n
        specified date as a datetime object.'''
        year = int(date_string[:4])
        month = int(date_string[5:7])
        day = int(date_string[8:])
        return pendulum.datetime(year, month, day)


In [59]:
def get_stop_details(gtfs_feed, route_id, service_date_string):

    service_date = string_to_datetime(service_date_string)

    # Get trip summary for service date using chn-ghost-buses make_trip_summary() function
    trip_summary = make_trip_summary(gtfs_feed, service_date, service_date)

    # filter down to the specified route
    trip_summary = trip_summary[trip_summary['route_id'] == route_id]

    # list trip ids for this route
    trip_list = trip_summary['trip_id'].unique().tolist()

    # get stop times data for the trips on this route
    stop_times = gtfs_feed.stop_times

    # filter stop times down to the relevant trips
    stop_times = stop_times.loc[stop_times['trip_id'].isin(trip_list)]

    # Add service id, route, and direction to the stop times data
    stop_times = stop_times.merge(trip_summary[['trip_id', 'route_id', 'service_id', 'direction', 'raw_date']], on='trip_id')

    # filter stop details down to the relevant route
    stop_times = stop_times.loc[stop_times['route_id'] == route_id]

    # Eliminate duplicates - every line shows up twice.
    # TODO:  Investigate why. Is this related to the calendar_cross line in the 
    # make_trip_summary() function? And/or the fact that I'm using the 
    # same date as start and end date as arguments in make_trip_summary?()
    stop_times = stop_times.drop_duplicates()

    return stop_times

stop_details = get_stop_details(gtfs_feed, route_id, service_date_string)
stop_details

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled,arrival_hour,departure_hour,route_id,service_id,direction,raw_date
0,6530000010020,04:37:30,04:37:30,10603,1,St Louis,0,0,4,4,55,65301,West,2023-01-09 00:00:00+00:00
1,6530000010020,04:37:55,04:37:55,10604,2,St Louis,0,639,4,4,55,65301,West,2023-01-09 00:00:00+00:00
2,6530000010020,04:38:31,04:38:31,10605,3,St Louis,0,1375,4,4,55,65301,West,2023-01-09 00:00:00+00:00
3,6530000010020,04:39:01,04:39:01,10606,4,St Louis,0,2061,4,4,55,65301,West,2023-01-09 00:00:00+00:00
4,6530000010020,04:39:32,04:39:32,10607,5,St Louis,0,2826,4,4,55,65301,West,2023-01-09 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26368,6530045537030,23:38:09,23:38:09,15366,63,Midway Orange Line,0,45089,23,23,55,65302,West,2023-01-09 00:00:00+00:00
26369,6530045537030,23:38:42,23:38:42,10632,64,Midway Orange Line,0,45760,23,23,55,65302,West,2023-01-09 00:00:00+00:00
26370,6530045537030,23:39:14,23:39:14,10633,65,Midway Orange Line,0,46457,23,23,55,65302,West,2023-01-09 00:00:00+00:00
26371,6530045537030,23:39:44,23:39:44,17030,66,Midway Orange Line,0,47108,23,23,55,65302,West,2023-01-09 00:00:00+00:00


In [60]:
# Get headways
def get_headways(stop_details):
    
    df_headways = pd.DataFrame()

    # consider each stop and direction of travel separately
    for stop_id, direction in set(zip(stop_details['stop_id'], stop_details['direction'])):

        # dataframe filtered to one stop_id and direction
        df = stop_details.loc[
            stop_details['stop_id'] == stop_id].loc[
                stop_details['direction'] == direction]

        # sort by arrival time
        df = df.sort_values('arrival_time')

        arrivals = pd.to_timedelta(df['arrival_time'])
        previous_arrivals = np.roll(arrivals, shift=1)

        # Calculate headways
        df['headway'] =  arrivals - previous_arrivals

        # Remove the first headway (no previous arrival time to compare with)
        df.iloc[0, -1] = None

        df_headways = pd.concat([df_headways, df])


    return df_headways

headways = get_headways(stop_details)

headways[headways['stop_id'] == '14122']


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled,arrival_hour,departure_hour,route_id,service_id,direction,raw_date,headway
25167,6530042199010,00:46:00,00:46:00,14122,67,Midway Orange Line,0,52206,0,0,55,65315,West,2023-01-09 00:00:00+00:00,NaT
23700,6530040471020,01:07:30,01:07:30,14122,67,Midway Orange Line,0,52206,1,1,55,65315,West,2023-01-09 00:00:00+00:00,0 days 00:21:30
3818,6530004981020,01:31:30,01:31:30,14122,67,Midway Orange Line,0,52206,1,1,55,65315,West,2023-01-09 00:00:00+00:00,0 days 00:24:00
15832,6530025242020,04:49:00,04:49:00,14122,67,Midway Orange Line,0,52206,4,4,55,65301,West,2023-01-09 00:00:00+00:00,0 days 03:17:30
15354,6530023086020,05:22:30,05:22:30,14122,67,Midway Orange Line,0,52206,5,5,55,65301,West,2023-01-09 00:00:00+00:00,0 days 00:33:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17898,6530028331030,22:12:00,22:12:00,14122,1,Museum of Science & Industry,0,0,22,22,55,65302,East,2023-01-09 00:00:00+00:00,0 days 00:19:00
20272,6530033437020,22:32:00,22:32:00,14122,1,Museum of Science & Industry,0,0,22,22,55,65301,East,2023-01-09 00:00:00+00:00,0 days 00:20:00
2228,6530003050020,22:54:30,22:54:30,14122,1,Museum of Science & Industry,0,0,22,22,55,65301,East,2023-01-09 00:00:00+00:00,0 days 00:22:30
10963,6530015704020,23:17:30,23:17:30,14122,1,Museum of Science & Industry,0,0,23,23,55,65301,East,2023-01-09 00:00:00+00:00,0 days 00:23:00


In [54]:
# Copied from actual_headways.ipynb

def get_headway_stats(headways:pd.DataFrame, headway_column_name:LineString) -> dict:
    '''Parameters:\n
    headways is a dataframe obtained using get_headways() or get_scheduled_headways().\n
    headway_column_name is the name of the column containing headways:  'est_headway' if these
    are based on actual bus times using get_headways() or 'scheduled_headway' if these are based
    on GTFS schedules using get_Scheduled_headways()\n
 
    Data returned:\n
    Statisics on the headways are returned as a dictionary.'''
    est_headways = headways[headway_column_name]
    stats = {
        'mean':est_headways.mean(),
        'max':est_headways.max(),
        'min':est_headways.min(),
        '25th_percentile':est_headways.quantile(0.25), # 25th percentile
        'median':est_headways.median(), # 50th percentile
        '75th_percentile':est_headways.quantile(0.75)
    }
    return stats

get_headway_stats(headways, 'headway')

{'mean': Timedelta('0 days 00:13:05.845286606'),
 'max': Timedelta('0 days 03:17:30'),
 'min': Timedelta('0 days 00:00:14'),
 '25th_percentile': Timedelta('0 days 00:09:00'),
 'median': Timedelta('0 days 00:12:00'),
 '75th_percentile': Timedelta('0 days 00:15:00')}